In [1]:
import tensorflow.keras
from tensorflow.keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential
import librosa
import librosa.display
import numpy as np
import pickle
import random
import glob
import re
from sklearn.preprocessing import LabelEncoder

### TODO: Talk about why 3 second time slots

In [2]:
# Load in datasets and combine into one
with open('../data/temp/dataset1.pkl', 'rb') as f:
    df1 = pickle.load(f)
    
with open('../data/temp/dataset2.pkl', 'rb') as f:
    df2 = pickle.load(f)

df = df1 + df2

In [3]:
frac = 0.80
split = int(np.floor(len(df) * frac))
random.shuffle(df)

train = df[:split]
test = df[split:]

_, X_train, y_train = zip(*train)
_, X_test, y_test = zip(*test)

### TODO Reshaping for RNN 

# Reshape for CNN input
X_train = np.array([x.reshape( (128, 128, 1) ) for x in X_train])
X_test = np.array([x.reshape( (128, 128, 1) ) for x in X_test])

# One-Hot encoding for classes
y_train = np.array(tensorflow.keras.utils.to_categorical(y_train, 4))
y_test = np.array(tensorflow.keras.utils.to_categorical(y_test, 4))

Specity, compile, and evaluate model

Model based on *Deep Convolutional Neural Networks and Data Augmentation for Environmental Sound Classification*

> arXiv: https://arxiv.org/pdf/1608.04363.pdf

Improvements:

*   Modify model architecture, loss, and optimizer to improve model
*   Assess model performance on a variety of measure appropriate for multinomial classification problem
*   Compare model performance with different inputs: percussive only, harmonics only, both (current)


In [16]:
model = Sequential()
input_shape=(128, 128, 1)

model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('sigmoid'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('sigmoid'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(Activation('sigmoid'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('sigmoid'))
model.add(Dropout(rate=0.5))

model.add(Dense(4))
model.add(Activation('softmax'))

In [18]:
model.compile(
	optimizer="Adam",
	loss="categorical_crossentropy",
	metrics=['accuracy'])

model.fit(
	x=X_train, 
	y=y_train,
    epochs=5,
    batch_size=128,
    validation_data= (X_test, y_test))

score = model.evaluate(
	x=X_test,
	y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 944 samples, validate on 236 samples
Epoch 1/5
640/944 [===================>..........] - ETA: 18s - loss: 1.5154 - acc: 0.2297

KeyboardInterrupt: 